# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    # YOUR CODE HERE
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    nhl_df = nhl_df.rename(columns={'team':'NHL'})
    nhl_df = nhl_df[nhl_df['year'] == 2018] #Deixar apenas times de 2018
    nhl_df = nhl_df[['NHL', 'W', 'L']] #Deixar apenas as colunas com times,vitórias e derrotas
    nhl_df["NHL"] = nhl_df["NHL"].str.replace(r'\*$', "")  # Remover * de todos times
    #Remover linhas desnecessárias:
    strings_a_remover = ['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division'] 
    nhl_df = nhl_df.loc[~nhl_df['NHL'].isin(strings_a_remover)]
    nhl_df['NHL'] = nhl_df['NHL'].str.split().str[-1] #Deixar apenas o nome do time (removendo o nome da cidade)

    cities = cities.drop(columns = ['NFL', 'MLB', 'NBA']) #Remover colunas desnecessárias
    cities.replace("—", pd.NA, inplace=True)  # Substituir "—" por Nan
    cities["NHL"] = cities["NHL"].str.replace(r'\[note \d+\]$', "")  # Remover [note {número}] de todos nomes de times   
    cities['NHL'] = cities['NHL'].str.split().str[-1] #Deixar apenas nome principal do time, assim, nhl_df e cities possuem equivalentes na coluna NHL
    
    df_merged = pd.merge(cities, nhl_df, on="NHL", how="right") #Juntar os dois dataframes pela coluna NHL
    #Adicionando cidade para os times que estavam na msm cidade (ex: ducks e kings são de LA, então apenas um deles foi colocado pelo merge em LA)
    df_merged['Metropolitan area'][14] = df_merged['Metropolitan area'][15] = 'New York City'
    df_merged['Metropolitan area'][26] = 'Los Angeles'
    #Transformando colunas com número de derrotas e vitórias para numérico
    df_merged['W'] = pd.to_numeric(df_merged['W'])
    df_merged['L'] = pd.to_numeric(df_merged['L'])
    
    #Juntando pela cidades, então times por exemplo, LA vai ter a soma das derrotas e vitórias de Ducks e Kings
    df_agrupadoVitorias = df_merged.groupby('Metropolitan area')['W'].sum().reset_index() 
    df_agrupadoDerrotas = df_merged.groupby('Metropolitan area')['L'].sum().reset_index()   
    df_agrupado = pd.merge(df_agrupadoVitorias, df_agrupadoDerrotas, on="Metropolitan area", how="inner")  #juntando as derrotas e vitórias 
    df_agrupado['ratio'] = df_agrupado['W']/(df_agrupado['W']+df_agrupado['L']) #criando win/loss ratio
    df_agrupado = pd.merge(df_agrupado, cities, on="Metropolitan area", how="left") #adicionando o win/loss ratio no dataframe cidades
    df_agrupado = df_agrupado.drop(columns = ['NHL'])
    
    population_by_region = pd.to_numeric(df_agrupado['Population (2016 est.)[8]']).tolist()
    win_loss_by_region = df_agrupado['ratio'].tolist()
    
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
nhl_correlation()

         Metropolitan area Population (2016 est.)[8]         NHL   W   L
0           Tampa Bay Area                   3032171   Lightning  54  23
1                   Boston                   4794447      Bruins  50  20
2                  Toronto                   5928040       Leafs  49  26
3    Miami–Fort Lauderdale                   6066387    Panthers  44  30
4                  Detroit                   4297617       Wings  30  39
5                 Montreal                   4098927   Canadiens  29  40
6                   Ottawa                   1323783    Senators  28  43
7                  Buffalo                   1132804      Sabres  25  45
8         Washington, D.C.                   6131977    Capitals  49  26
9               Pittsburgh                   2342299    Penguins  47  29
10            Philadelphia                   6070500      Flyers  42  26
11                Columbus                   2041520     Jackets  45  30
12           New York City                  2015363

0.01230899645574425

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    esporte = 'NBA'
    
    nba_df = nba_df.rename(columns={'team':esporte})
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df = nba_df[[esporte, 'W', 'L']]
    nba_df[esporte] = nba_df[esporte].str.replace(r'\s*\(\d+\)$|\s*\*\s*\(\d+\)$', "")  # Remover * de todos times
    nba_df[esporte] = nba_df[esporte].str.split().str[-1]
    
    cities = cities.drop(columns = ['NFL', 'MLB', 'NHL'])
    cities.replace("—", pd.NA, inplace=True)  # Substituir "..." por Nan
    cities[esporte] = cities[esporte].str.replace(r'\[note \d+\]$', "")  # Remover [note {número}] de todos nomes de times    
    cities[esporte] = cities[esporte].str.split().str[-1]
    
    df_merged = pd.merge(cities, nba_df, on=esporte, how="right")
    df_merged['Metropolitan area'][10] =  'New York City'
    df_merged['Metropolitan area'][25] = 'Los Angeles'
    df_merged['W'] = pd.to_numeric(df_merged['W'])
    df_merged['L'] = pd.to_numeric(df_merged['L'])
    
    df_agrupadoVitorias = df_merged.groupby('Metropolitan area')['W'].sum().reset_index()
    df_agrupadoDerrotas = df_merged.groupby('Metropolitan area')['L'].sum().reset_index()   
    df_agrupado = pd.merge(df_agrupadoVitorias, df_agrupadoDerrotas, on="Metropolitan area", how="inner")
    df_agrupado['ratio'] = df_agrupado['W']/(df_agrupado['W']+df_agrupado['L'])
    df_agrupado = pd.merge(df_agrupado, cities, on="Metropolitan area", how="left")
    df_agrupado = df_agrupado.drop(columns = [esporte])
    
    population_by_region = pd.to_numeric(df_agrupado['Population (2016 est.)[8]']).tolist()
    win_loss_by_region = df_agrupado['ratio'].tolist()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
nba_correlation()

(-0.17657160252844614, 0.36874741604463)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [84]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    esporte = 'MLB'
    
    mlb_df = mlb_df.rename(columns={'team':esporte})
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df = mlb_df[[esporte, 'W', 'L', 'W-L%']]
    mlb_df[esporte][0] = 'Boston RedSox'
    mlb_df[esporte][8] = 'Chicago WhiteSox'
    mlb_df[esporte] = mlb_df[esporte].str.split().str[-1]
    
    
    cities = cities.drop(columns = ['NFL', 'NBA', 'NHL'])
    cities.replace("—", pd.NA, inplace=True)  # Substituir "..." por Nan
    cities[esporte] = cities[esporte].str.replace(r'\[note \d+\]$', "")  # Remover [note {número}] de todos nomes de times    
    cities[esporte][3] = 'Chicago WhiteSox'
    cities[esporte][7] = 'Boston RedSox'
    cities[esporte] = cities[esporte].str.split().str[-1]
    
    df_merged = pd.merge(cities, mlb_df, on=esporte, how="right")
    df_merged['Metropolitan area'][1] =  'New York City'
    df_merged['Metropolitan area'][21] =  'Chicago'
    df_merged['Metropolitan area'][25] = 'Los Angeles'
    df_merged['Metropolitan area'][28] = 'San Francisco Bay Area'
    df_merged['W'] = pd.to_numeric(df_merged['W'])
    df_merged['L'] = pd.to_numeric(df_merged['L'])
    df_merged['W-L%'] = pd.to_numeric(df_merged['W-L%'])
    
    df_agrupadoVitorias = df_merged.groupby('Metropolitan area')['W'].sum().reset_index()
    df_agrupadoDerrotas = df_merged.groupby('Metropolitan area')['L'].sum().reset_index()   
    df_agrupado = pd.merge(df_agrupadoVitorias, df_agrupadoDerrotas, on="Metropolitan area", how="inner")
    df_agrupado = df_merged.groupby('Metropolitan area')['W-L%'].median().reset_index()   
    #df_agrupado['ratio'] = df_agrupado['W']/(df_agrupado['W']+df_agrupado['L'])
    df_agrupado = pd.merge(df_agrupado, cities, on="Metropolitan area", how="left")
    df_agrupado = df_agrupado.drop(columns = [esporte])
    
    population_by_region = pd.to_numeric(df_agrupado['Population (2016 est.)[8]']).tolist()
    win_loss_by_region = df_agrupado['ratio'].tolist()
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
mlb_correlation()

0.15003737475409498

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [42]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    esporte = 'NFL'
    
    nfl_df = nfl_df.rename(columns={'team':esporte})
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[[esporte, 'W', 'L']]
    nfl_df[esporte] = nfl_df[esporte].str.replace(r'\*$|\+$', "")
    nfl_df = nfl_df[~nfl_df[esporte].str.startswith(('AFC', 'NFC'))]
    nfl_df[esporte] = nfl_df[esporte].str.split().str[-1]
    
    cities = cities.drop(columns = ['MLB', 'NBA', 'NHL'])
    cities.replace("—", pd.NA, inplace=True)  # Substituir "..." por Nan
    cities[esporte] = cities[esporte].str.replace(r'\[note \d+\]$', "")  # Remover [note {número}] de todos nomes de times    
    cities[esporte] = cities[esporte].str.split().str[-1]
    
    df_merged = pd.merge(cities, nfl_df, on=esporte, how="right")
    df_merged['Metropolitan area'][19] =  'New York City'
    df_merged['Metropolitan area'][28] = 'Los Angeles'
    df_merged['Metropolitan area'][30] = 'San Francisco Bay Area' 
    df_merged['W'] = pd.to_numeric(df_merged['W'])
    df_merged['L'] = pd.to_numeric(df_merged['L'])
    
    df_agrupadoVitorias = df_merged.groupby('Metropolitan area')['W'].sum().reset_index()
    df_agrupadoDerrotas = df_merged.groupby('Metropolitan area')['L'].sum().reset_index()   
    df_agrupado = pd.merge(df_agrupadoVitorias, df_agrupadoDerrotas, on="Metropolitan area", how="inner")
    df_agrupado['ratio'] = df_agrupado['W']/(df_agrupado['W']+df_agrupado['L'])
    df_agrupado = pd.merge(df_agrupado, cities, on="Metropolitan area", how="left")
    df_agrupado = df_agrupado.drop(columns = [esporte])
    
    population_by_region = pd.to_numeric(df_agrupado['Population (2016 est.)[8]']).tolist()
    win_loss_by_region = df_agrupado['ratio'].tolist()

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

(0.004922112149349409, 0.9797833458363694)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values